In [42]:
import numpy as np
import pandas as pd
import gurobipy as gp
from tqdm import tqdm
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# Reading the files
stocks2019 = pd.read_csv('stocks2019.csv')
stocks2020 = pd.read_csv('stocks2020.csv')

## Question 1

In [5]:
# Procxessing the data to get the the attributes and features:
# Percentage Change, NDX change, correlation matrix
stocks2019_cols = list(stocks2019.columns)
stocks2019_change = stocks2019[stocks2019_cols[2:]].pct_change().dropna()
q2019 = stocks2019[['NDX']].pct_change().dropna()
stocks2019_corr = stocks2019_change.corr()
display(stocks2019_corr)
stocks2019_index = q2019.index.values.tolist()

stocks2020_cols = list(stocks2020.columns)
stocks2020_change = stocks2020[stocks2020_cols[2:]].pct_change().dropna()
q2020 = stocks2020[['NDX']].pct_change().dropna()
stocks2020_corr = stocks2020_change.corr()
display(stocks2020_corr)
stocks2020_index = q2020.index.values.tolist()

,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
ATVI,1.000000,0.399939,0.365376,0.223162,0.216280,0.433097,0.426777,0.467076,0.203956,0.329355,...,0.322906,0.128241,0.464850,0.316549,0.259679,0.218149,0.311659,0.303077,0.043389,0.249667
ADBE,0.399939,1.000000,0.452848,0.368928,0.363370,0.552125,0.540404,0.598237,0.291978,0.473815,...,0.360392,0.201151,0.711339,0.541243,0.402171,0.228106,0.650430,0.361516,0.207403,0.289497
AMD,0.365376,0.452848,1.000000,0.301831,0.344252,0.418861,0.417254,0.549302,0.151452,0.503733,...,0.332776,0.210623,0.498342,0.330900,0.272983,0.281950,0.407626,0.438892,0.017283,0.478010
ALXN,0.223162,0.368928,0.301831,1.000000,0.332433,0.315993,0.307698,0.363170,0.342022,0.317040,...,0.257143,0.408936,0.350581,0.191489,0.522423,0.192720,0.416396,0.289908,0.047947,0.200356
ALGN,0.216280,0.363370,0.344252,0.332433,1.000000,0.248747,0.250316,0.399281,0.264599,0.328280,...,0.175957,0.128559,0.360886,0.251855,0.334978,0.219595,0.308968,0.284407,0.088059,0.253934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WBA,0.218149,0.228106,0.281950,0.192720,0.219595,0.232900,0.230603,0.288168,0.194490,0.347861,...,0.257049,0.145330,0.195475,0.126118,0.214585,1.000000,0.211641,0.296309,-0.130798,0.276643
WDAY,0.311659,0.650430,0.407626,0.416396,0.308968,0.379493,0.371826,0.424748,0.211712,0.351734,...,0.235045,0.269545,0.569672,0.401476,0.376047,0.211641,1.000000,0.315699,0.209240,0.254117
WDC,0.303077,0.361516,0.438892,0.289908,0.284407,0.328619,0.322110,0.419620,0.172623,0.602935,...,0.377215,0.126463,0.331916,0.308558,0.314773,0.296309,0.315699,1.000000,-0.076115,0.516958
XEL,0.043389,0.207403,0.017283,0.047947,0.088059,0.059930,0.052570,0.076724,0.137857,-0.047259,...,-0.172752,0.074686,0.280371,0.407770,0.191667,-0.130798,0.209240,-0.076115,1.000000,-0.117552


,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
ATVI,1.000000,0.730174,0.579314,0.403073,0.237541,0.567828,0.566060,0.626415,0.565433,0.454953,...,0.154898,0.120848,0.603622,0.497564,0.579678,0.247209,0.410701,0.280593,0.411515,0.479460
ADBE,0.730174,1.000000,0.659260,0.491232,0.437418,0.798263,0.795778,0.753873,0.620782,0.720870,...,0.344892,0.367310,0.754475,0.672608,0.602576,0.322319,0.709340,0.488606,0.520639,0.672700
AMD,0.579314,0.659260,1.000000,0.395293,0.396404,0.604720,0.603124,0.621273,0.509703,0.599515,...,0.292341,0.362999,0.669125,0.589336,0.538653,0.285847,0.507533,0.433935,0.469962,0.609899
ALXN,0.403073,0.491232,0.395293,1.000000,0.327819,0.434124,0.430771,0.426561,0.524339,0.457500,...,0.332673,0.281982,0.506699,0.473451,0.457318,0.330271,0.353305,0.348578,0.373230,0.475345
ALGN,0.237541,0.437418,0.396404,0.327819,1.000000,0.547835,0.540729,0.304170,0.390302,0.589460,...,0.454142,0.568183,0.527947,0.506896,0.307963,0.372854,0.397577,0.543233,0.464390,0.469649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WBA,0.247209,0.322319,0.285847,0.330271,0.372854,0.452024,0.451883,0.247289,0.483689,0.378966,...,0.382775,0.356305,0.373740,0.430280,0.309819,1.000000,0.151933,0.413085,0.471935,0.366630
WDAY,0.410701,0.709340,0.507533,0.353305,0.397577,0.621941,0.617913,0.533779,0.355236,0.584594,...,0.405752,0.486523,0.552892,0.560405,0.397599,0.151933,1.000000,0.528519,0.376101,0.475497
WDC,0.280593,0.488606,0.433935,0.348578,0.543233,0.564265,0.556093,0.335195,0.422348,0.700760,...,0.568574,0.657512,0.538575,0.567524,0.372172,0.413085,0.528519,1.000000,0.474558,0.523815
XEL,0.411515,0.520639,0.469962,0.373230,0.464390,0.563182,0.552362,0.379968,0.616083,0.572824,...,0.237195,0.460488,0.676772,0.752317,0.456288,0.471935,0.376101,0.474558,1.000000,0.459359


In [6]:
# Processing the features extracted above
n_stocks_2019 = len(stocks2019_corr)
stocks2019_names = list(stocks2019_corr.columns)
stocks2019_corr_dict = stocks2019_corr.to_dict()
stocks2019_returns_dict = stocks2019_change.to_dict()
stocks2019_dates = list(stocks2019[stocks2019_cols[0]])

n_stocks_2020 = len(stocks2020_corr)
stocks2020_names = list(stocks2020_corr.columns)
stocks2020_corr_dict = stocks2020_corr.to_dict()
stocks2020_returns_dict = stocks2020_change.to_dict()
stocks2020_dates = list(stocks2020[stocks2020_cols[0]])

## Question 2

In [7]:
def stockSelection(m, stock_names, correlation_dict): # Function to select the stocks in the 2_step prcess
    # Some minute sanity checks
    assert m <= len(stocks2019_names)
    assert len(stocks2019_names) <= len(correlation_dict)

    stock_selection_model = gp.Model()
    stock_selection_model.ModelSense = gp.GRB.MAXIMIZE

    x = stock_selection_model.addVars(stock_names,stock_names,vtype='B') # similarity variable
    y = stock_selection_model.addVars(stock_names,vtype='B') # limiting variable

    # Constraint to select just m stocks
    sum_y = 0
    for i in stock_names:
        sum_y += y[i]
    stock_selection_model.addConstr(sum_y == m)

    # Constraint to get similar stocks
    for i in stock_names:
        sum_x = 0
        for j in stock_names:
            sum_x += x[i, j]
        stock_selection_model.addConstr( sum_x == 1)

    # Constraint to deselect similar stock
    for i in stock_names:
        for j in stock_names:
            stock_selection_model.addConstr(x[i, j] <= y[j])

    # Setting the objective function to maximize the correlation
    sum_obj = 0
    for i in stock_names:
        for j in stock_names:
            sum_obj += correlation_dict[i][j] * x[i, j]
    stock_selection_model.setObjective(sum_obj)

    # Setting model parameters
    stock_selection_model.Params.OutputFlag = 0
    stock_selection_model.optimize()
    
    # Converting selected stocks to a list
    selectedStocks = []
    for i in stock_names:
        for j in stock_names:
            if x[i,j].X == 1 and j not in selectedStocks:
                selectedStocks.append(j)
                
    return stock_selection_model, selectedStocks

In [6]:
# Testing stockSelection for m=5
selection_model, selectedStocks = stockSelection(m=5, stock_names=stocks2019_names, correlation_dict=stocks2019_corr_dict)
print(selection_model.ObjVal)
print(selectedStocks)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
54.83990652229108
['MSFT', 'VRTX', 'MXIM', 'LBTYK', 'XEL']


In [11]:
def portfolioWeights(m,selectedStocks, stock_names, stocks_indices_old, stock_indices_new, \
                     stock_returns_old, stock_returns_new, q, q_new, timeLimit = 300, save=False): # Function to get the model weight (2-steo)
    # Minute sanity checks
    assert len(q.columns) == 1
    assert len(q_new.columns) == 1
    
    # Converting actual NDX returns provided to a list
    q = list(q[list(q.columns)[0]].values)
    q_new = list(q_new[list(q_new.columns)[0]].values)

    # Intializing the model and setting the model parameters
    weight_model = gp.Model()
    weight_model.ModelSense = gp.GRB.MINIMIZE
    weight_model.Params.OutputFlag = 0
    weight_model.Params.TIME_LIMIT = timeLimit

    # Decision Variables
    weight_SelectedStocks = weight_model.addVars(selectedStocks, ub=[1]*len(selectedStocks)) # variable for weight of selected stock
    u = weight_model.addVars(stocks_indices_old) # dummy variable to solve non-linearities

    # Constraint for sum of weight = 1
    sum_weighgts = 0
    for i in selectedStocks:
        sum_weighgts += weight_SelectedStocks[i]
    weight_model.addConstr(sum_weighgts == 1)

    # Constraints to minimize non linearity like min -> |X-1|
    for j in stocks_indices_old:
        sum_weight_return = 0
        for i in selectedStocks:
            sum_weight_return += weight_SelectedStocks[i] * stock_returns_old[i][j]
        weight_model.addConstr(u[j] >= (q[j-1] - sum_weight_return))
        weight_model.addConstr(u[j] >= -(q[j-1] - sum_weight_return))
    
    # Setting the objective to minimize the absolute errors
    sum_obj = 0 
    for j in stocks_indices_old:
        sum_obj += u[j]
    weight_model.setObjective(sum_obj)

    weight_model.optimize()        

    # Calculating returns from model for the old data
    returns_old = []
    for j in stocks_indices_old:
        sum_return = 0
        for i in selectedStocks:
            sum_return += weight_SelectedStocks[i].X * stock_returns_old[i][j]
        returns_old.append(sum_return)

    # Calculating returns from model for the new data
    returns_new = []
    for j in stock_indices_new:
        sum_return = 0
        for i in selectedStocks:
            sum_return += weight_SelectedStocks[i].X * stock_returns_new[i][j]
        returns_new.append(sum_return)
    
    # Saving the calculated returns as per user's demand
    if save:
        returns_old_save = []
        for j in stocks_indices_old:
            sum_return = 0
            for i in selectedStocks:
                sum_return += weight_SelectedStocks[i].X * stock_returns_old[i][j]
            returns_old_save.append([j, q[j-1], sum_return])
        returns_old_save = pd.DataFrame(returns_old_save, columns=['Index', 'Index Return', 'Portfolio Return'])
        returns_old_save.to_csv(f'Returns_old_m-{m}_2step.csv', index=False)

        returns_new_save = []
        for j in stock_indices_new:
            sum_return = 0
            for i in selectedStocks:
                sum_return += weight_SelectedStocks[i].X * stock_returns_new[i][j]
            returns_new_save.append([j, q_new[j-1], sum_return])
        returns_new_save = pd.DataFrame(returns_new_save, columns=['Index', 'Index Return', 'Portfolio Return'])
        returns_new_save.to_csv(f'Returns_new_m-{m}_2step.csv', index=False)
    
    # Saving the weights in a dataframe
    selectedStocks_weights = pd.DataFrame(index=stock_names)
    selectedStocks_weights[f'm_{m}'] = np.nan
    for i in selectedStocks:
        selectedStocks_weights[f'm_{m}'].loc[i] = weight_SelectedStocks[i].X
    selectedStocks_weights = selectedStocks_weights[selectedStocks_weights[f'm_{m}'].notna()]
        
    # Evaluating the model on the old and the new data
    model_error = weight_model.objVal
    new_return_error = sum(abs(np.array(q_new) - np.array(returns_new)))
    old_return_error_rmse = np.sqrt(np.mean((np.array(q) - np.array(returns_old))**2))
    new_return_error_rmse = np.sqrt(np.mean((np.array(q_new) - np.array(returns_new))**2))
        
    return weight_model, selectedStocks_weights, model_error, old_return_error_rmse, new_return_error, new_return_error_rmse


def portfolioWeights_MIP(m, stock_names, stocks_indices_old, stock_indices_new, \
                     stock_returns_old, stock_returns_new, q, q_new, timeLimit = 300, save=False): # Function to get the model weight (MIP)
    # Minute sanity checks
    assert len(q.columns) == 1
    assert len(q_new.columns) == 1

    # Converting actual NDX returns provided to a list
    q = list(q[list(q.columns)[0]].values)
    q_new = list(q_new[list(q_new.columns)[0]].values)
    
    # Intializing the model and setting the model parameters
    MIP_model = gp.Model()
    MIP_model.ModelSense = gp.GRB.MINIMIZE
    MIP_model.Params.OutputFlag = 0
    MIP_model.Params.TIME_LIMIT = timeLimit
    bigM = 1

    # Decision Variables
    weights = MIP_model.addVars(stock_names) # variable for weight of selected stock
    y = MIP_model.addVars(stock_names,vtype='B') # variable to limit the weights or push them to zero - BigM constraint
    u = MIP_model.addVars(stocks_indices_old) # dummy variable to solve non-linearities

    # Constraint for sum of weight = 1
    sum_weights = 0
    for i in stock_names:
        sum_weights += weights[i]
    MIP_model.addConstr(sum_weights == 1)

    # Constraints to minimize non linearity like min -> |X-1|
    for j in stocks_indices_old:
        sum_return = 0
        for i in stock_names:
            sum_return += weights[i]*stock_returns_old[i][j]
        MIP_model.addConstr(u[j] >=  (q[j-1] - sum_return))
        MIP_model.addConstr(u[j] >=  -(q[j-1] - sum_return))
    
    # Setting the bigM constraints
    for i in stock_names:
        MIP_model.addConstr(weights[i] <= y[i]*bigM)

    # Constraint to select the number of constraints specified by the user
    sum_y = 0
    for i in stock_names:
        sum_y += y[i]
    MIP_model.addConstr(sum_y == m)

    # Setting the objective to minimize the absolute errors
    sum_obj = 0
    for j in stocks_indices_old:
        sum_obj += u[j]
    MIP_model.setObjective(sum_obj)
    
    MIP_model.optimize()
    
    # Calculating returns from model for the old data
    returns_old = []
    for j in stocks_indices_old:
        sum_return = 0
        for i in stock_names:
            sum_return += weights[i].X * stock_returns_old[i][j]
        returns_old.append(sum_return)

    # Calculating returns from model for the new data
    returns_new = []
    for j in stock_indices_new:
        sum_return = 0
        for i in stock_names:
            sum_return += weights[i].X * stock_returns_new[i][j]
        returns_new.append(sum_return)  
    
    # Saving the calculated returns as per user's demand
    if save:
        returns_old_save = []
        for j in stocks_indices_old:
            sum_return = 0
            for i in stock_names:
                sum_return += weights[i].X * stock_returns_old[i][j]
            returns_old_save.append([j, q[j-1], sum_return])
        returns_old_save = pd.DataFrame(returns_old_save, columns=['Index', 'Index Return', 'Portfolio Return'])
        returns_old_save.to_csv(f'Returns_old_m-{m}_T-{timeLimit}.csv', index=False)

        returns_new_save = []
        for j in stock_indices_new:
            sum_return = 0
            for i in stock_names:
                sum_return += weights[i].X * stock_returns_new[i][j]
            returns_new_save.append([j, q_new[j-1], sum_return])
        returns_new_save = pd.DataFrame(returns_new_save, columns=['Index', 'Index Return', 'Portfolio Return'])
        returns_new_save.to_csv(f'Returns_new_m-{m}_T-{timeLimit}.csv', index=False)

    # Saving the weights in a dataframe
    stockWeights = pd.DataFrame(index=stock_names)
    stockWeights[f'm_{m}'] = np.nan
    for i in stock_names:
        stockWeights[f'm_{m}'].loc[i] = weights[i].X
    stockWeights = stockWeights[stockWeights[f'm_{m}'].notna()]
        
    # Evaluating the model on the old and the new data
    model_error = MIP_model.objVal
    new_return_error = sum(abs(np.array(q_new) - np.array(returns_new)))
    old_return_error_rmse = np.sqrt(np.mean((np.array(q) - np.array(returns_old))**2))
    new_return_error_rmse = np.sqrt(np.mean((np.array(q_new) - np.array(returns_new))**2))
        
    return MIP_model, stockWeights, model_error, old_return_error_rmse, new_return_error, new_return_error_rmse

In [8]:
# Getting portfolio weights for the selected 5 stocks
weight_model, weight, model_error, old_return_rmse, new_return_error, new_return_rmse = portfolioWeights(m=5, \
                                                                       selectedStocks=selectedStocks, \
                                                                        stock_names=stocks2019_names, \
                                                                        stocks_indices_old=stocks2019_index, \
                                                                        stock_indices_new=stocks2020_index, \
                                                                        stock_returns_old=stocks2019_returns_dict, \
                                                                        stock_returns_new=stocks2020_returns_dict, \
                                                                        q=q2019, \
                                                                        q_new=q2020, \
                                                                        timeLimit=600,
                                                                        save=True)
print(model_error)
print(new_return_error)
print(old_return_rmse)
print(new_return_rmse)
display(weight)

0.7891782824631471
1.1124373455076464
0.004399606152430161
0.006248354722298066


,m_5
LBTYK,0.048862
MXIM,0.210388
MSFT,0.580352
VRTX,0.071190
XEL,0.089208


## Question 3

In [9]:
# Running the 2-step approach for a wide array of selected stocks and saving the results
tries = []
i = 5
while i <= len(stocks2019_names):
    tries.append(i)
    i += 5

data = []
for try_ in tries:
    row = []
    row.append(try_)

    selection_model, selectedStocks = stockSelection(m=try_, stock_names=stocks2019_names, correlation_dict=stocks2019_corr_dict)
    row.append(len(selectedStocks))
    row.append(selection_model.ObjVal)

    weight_model, weight, model_error, old_return_rmse, new_return_error, new_return_rmse = portfolioWeights(m=try_, \
                                                                                                selectedStocks=selectedStocks, \
                                                                                                stock_names=stocks2019_names, \
                                                                                                stocks_indices_old=stocks2019_index, \
                                                                                                stock_indices_new=stocks2020_index, \
                                                                                                stock_returns_old=stocks2019_returns_dict, \
                                                                                                stock_returns_new=stocks2020_returns_dict, \
                                                                                                q=q2019, \
                                                                                                q_new=q2020, \
                                                                                                timeLimit=600,
                                                                                                save=False)
    
    row.append(model_error)
    row.append(old_return_rmse)
    row.append(new_return_error)
    row.append(new_return_rmse)

    data.append(row)

cols = ['m', '#StocksSelected', 'Selection_Obj_Val', '2019_return_error', '2019_return_RMSE', '2020_return_error', '2020_return_RMSE']
data = pd.DataFrame(data, columns=cols)
data.to_csv("stockSelectionApproach.csv", index=False)
data

,m,#StocksSelected,Selection_Obj_Val,2019_return_error,2019_return_RMSE,2020_return_error,2020_return_RMSE
0,5,5,54.839907,0.789178,0.004400,1.112437,0.006248
1,10,10,59.331930,0.701218,0.003790,1.102404,0.006221
2,15,15,63.268591,0.582276,0.003218,1.264689,0.007145
3,20,20,66.648075,0.478836,0.002653,0.899598,0.005088
4,25,25,69.764755,0.442966,0.002472,0.805065,0.004772
5,30,30,72.696797,0.418015,0.002399,0.769110,0.004666
6,35,35,75.505895,0.382997,0.002185,0.766774,0.004419
7,40,40,78.259157,0.370517,0.002157,0.791047,0.004471
8,45,45,80.892084,0.343944,0.002102,0.760495,0.004267
9,50,50,83.316268,0.332540,0.001968,0.772100,0.004311


## Question 4

In [10]:
# Running the MIP approach with a time limit of 20 secs for a wide array of selected stocks and saving the results
tries = []
i = 5
while i <= len(stocks2019_names):
    tries.append(i)
    i += 5

# tries = [90, 95]
data = []
for try_ in (pbar:= tqdm(tries)):
    pbar.set_description(f"Calculating for m = {try_}")
    row = []
    row.append(try_)
    
    weight_model, weight, model_error, old_return_rmse, new_return_error, new_return_rmse = portfolioWeights_MIP(m=try_, \
                                                                            stock_names=stocks2019_names, \
                                                                            stocks_indices_old=stocks2019_index, \
                                                                            stock_indices_new=stocks2020_index, \
                                                                            stock_returns_old=stocks2019_returns_dict, \
                                                                            stock_returns_new=stocks2020_returns_dict, \
                                                                            q=q2019, \
                                                                            q_new=q2020, \
                                                                            timeLimit=20,
                                                                            save=False)
    
    weight = weight.loc[weight[f"m_{try_}"] > 0]
    
    row.append(len(weight))
    row.append(model_error)
    row.append(old_return_rmse)
    row.append(new_return_error)
    row.append(new_return_rmse)

    data.append(row)

cols = ['m', '#StocksSelected', '2019_return_error', '2019_return_RMSE', '2020_return_error', '2020_return_RMSE']
data = pd.DataFrame(data, columns=cols)
data.to_csv("MIPApproach_T-20.csv", index=False)
data

Calculating for m = 100: 100%|██████████| 20/20 [05:22<00:00, 16.14s/it]


,m,#StocksSelected,2019_return_error,2019_return_RMSE,2020_return_error,2020_return_RMSE
0,5,5,0.499259,0.002859,0.777362,0.004181
1,10,10,0.304668,0.001663,0.640518,0.003533
2,15,15,0.234798,0.001322,0.698479,0.003699
3,20,21,0.209536,0.001216,0.608830,0.003294
4,25,25,0.235961,0.001332,0.655472,0.003458
5,30,29,0.182629,0.000994,0.485423,0.002605
6,35,35,0.172666,0.000975,0.476920,0.002570
7,40,39,0.135122,0.000782,0.411286,0.002287
8,45,45,0.103895,0.000612,0.378738,0.002080
9,50,50,0.072081,0.000445,0.379610,0.002071


In [13]:
# A sample run for somparison with MIP approach with a timelimit of 3600 sec
weight_model, weight, model_error, old_return_rmse, new_return_error, new_return_rmse = portfolioWeights_MIP(m=5, \
                                                                        stock_names=stocks2019_names, \
                                                                        stocks_indices_old=stocks2019_index, \
                                                                        stock_indices_new=stocks2020_index, \
                                                                        stock_returns_old=stocks2019_returns_dict, \
                                                                        stock_returns_new=stocks2020_returns_dict, \
                                                                        q=q2019, \
                                                                        q_new=q2020, \
                                                                        timeLimit=3600,
                                                                        save=True)
print(model_error)
print(old_return_rmse)
print(new_return_error)
print(new_return_rmse)
display(weight)

0.49925869205463597
0.0028589246578478284
0.7773624845503464
0.00418077151184521


,m_5
ATVI,0.0
ADBE,0.0
AMD,0.0
ALXN,0.0
ALGN,0.0
...,...
WBA,0.0
WDAY,0.0
WDC,0.0
XEL,0.0


In [20]:
# A sample run for somparison with MIP approach with a timelimit of 3600 sec
weight_model, weight, model_error, old_return_rmse, new_return_error, new_return_rmse = portfolioWeights_MIP(m=100, \
                                                                        stock_names=stocks2019_names, \
                                                                        stocks_indices_old=stocks2019_index, \
                                                                        stock_indices_new=stocks2020_index, \
                                                                        stock_returns_old=stocks2019_returns_dict, \
                                                                        stock_returns_new=stocks2020_returns_dict, \
                                                                        q=q2019, \
                                                                        q_new=q2020, \
                                                                        timeLimit=3600,
                                                                        save=True)
print(model_error)
print(old_return_rmse)
print(new_return_error)
print(new_return_rmse)
display(weight)
weight.to_csv('weights_m-100_T-3600.csv', index=True)

0.0449108164083929
0.0002989428739374261
0.36867085361031715
0.0020819640068757868


,m_100
ATVI,0.004223
ADBE,0.017138
AMD,0.004037
ALXN,0.002450
ALGN,0.000000
...,...
WBA,0.009130
WDAY,0.002555
WDC,0.000000
XEL,0.003662


In [11]:
# Running the MIP approach with a time limit of 3600 secs for a wide array of selected stocks and saving the results
tries = []
i = 5
while i <= len(stocks2019_names):
    tries.append(i)
    i += 5

# tries = [90, 95]
data = []
for try_ in (pbar:= tqdm(tries)):
    pbar.set_description(f"Calculating for m = {try_}")
    row = []
    row.append(try_)
    
    weight_model, weight, model_error, old_return_rmse, new_return_error, new_return_rmse = portfolioWeights_MIP(m=try_, \
                                                                            stock_names=stocks2019_names, \
                                                                            stocks_indices_old=stocks2019_index, \
                                                                            stock_indices_new=stocks2020_index, \
                                                                            stock_returns_old=stocks2019_returns_dict, \
                                                                            stock_returns_new=stocks2020_returns_dict, \
                                                                            q=q2019, \
                                                                            q_new=q2020, \
                                                                            timeLimit=3600,
                                                                            save=False)
    
    weight = weight.loc[weight[f"m_{try_}"] > 0]
    
    row.append(len(weight))
    row.append(model_error)
    row.append(old_return_rmse)
    row.append(new_return_error)
    row.append(new_return_rmse)

    data.append(row)

cols = ['m', '#StocksSelected', '2019_return_error', '2019_return_RMSE', '2020_return_error', '2020_return_RMSE']
data = pd.DataFrame(data, columns=cols)
data.to_csv("MIPApproach_T-3600.csv", index=False)
data

Calculating for m = 100: 100%|██████████| 20/20 [13:27:04<00:00, 2421.20s/it] 


,m,#StocksSelected,2019_return_error,2019_return_RMSE,2020_return_error,2020_return_RMSE
0,5,5,0.499259,0.002859,0.777362,0.004181
1,10,10,0.290137,0.001564,0.753372,0.004347
2,15,15,0.207520,0.001198,0.636754,0.003357
3,20,20,0.157501,0.000914,0.559464,0.002906
4,25,25,0.133442,0.000776,0.478015,0.002620
5,30,30,0.105653,0.000616,0.491623,0.002564
6,35,35,0.092986,0.000565,0.476436,0.002609
7,40,40,0.078577,0.000455,0.408250,0.002210
8,45,45,0.067697,0.000436,0.406685,0.002198
9,50,50,0.060979,0.000379,0.391677,0.002157


## Qestion 5 - Analysis

#### 2-Step Approach

In [36]:
df_returns_old = pd.read_csv('Returns_old_m-5_2step.csv')
fig = px.line(df_returns_old, x='Index', y=['Index Return', 'Portfolio Return'], title='Portfolio vs Index Returns: 2019')
fig.update_layout(
    legend=dict(
        orientation="h",  # Set orientation to horizontal
        yanchor="bottom",  # Anchor the legend to the bottom
        y=1.02  # Adjust the position of the legend
    )
)
fig.show()

df_returns_new = pd.read_csv('Returns_new_m-5_2step.csv')
df_returns_new['Error'] = df_returns_new['Index Return'] - df_returns_new['Portfolio Return']
fig = px.line(df_returns_new, x='Index', y=['Index Return', 'Portfolio Return'], title='Portfolio vs Index Returns: 2020')
fig.update_layout(
    legend=dict(
        orientation="h",  # Set orientation to horizontal
        yanchor="bottom",  # Anchor the legend to the bottom
        y=1.02  # Adjust the position of the legend
    )
)
fig.show()

fig = px.bar(df_returns_new, x='Index', y=['Error'], title='Error Rate 2020')
fig.show()

In [55]:
data_2_step = pd.read_csv('stockSelectionApproach.csv')
data_2_step['Delta'] = data_2_step['2020_return_error'] - data_2_step['2019_return_error']
fig1 = px.line(data_2_step, x='m', y=['2019_return_error', '2020_return_error'], title='Error Comparison: 2019 vs 2020')

data_MIP = pd.read_csv('MIPApproach_T-3600.csv')
data_MIP['Delta'] = data_MIP['2020_return_error'] - data_MIP['2019_return_error']
fig2 = px.line(data_MIP, x='m', y=['2019_return_error', '2020_return_error'], title='Error Comparison: 2019 vs 2020')

fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
fig2.update_traces(showlegend=False)
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig1.data[1], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)
fig.add_trace(fig2.data[1], row=1, col=2)
fig.update_layout(
    legend=dict(
        orientation="h",  # Set orientation to horizontal,
        yanchor="bottom",  # Anchor the legend to the bottom
        y=1.02  # Adjust the position of the legend
    ),
    title='Error Comparison'
    # width=600,  # Set the width of the figure
    # height=400  # Set the height of the figure
)
fig.show()

#### MIP Approach

In [37]:
df_returns_old = pd.read_csv('Returns_old_m-5_T-3600.csv')
fig = px.line(df_returns_old, x='Index', y=['Index Return', 'Portfolio Return'], title='Portfolio vs Index Returns: 2019')
fig.show()

df_returns_new = pd.read_csv('Returns_new_m-5_T-3600.csv')
df_returns_new['Error'] = df_returns_new['Index Return'] - df_returns_new['Portfolio Return']
fig = px.line(df_returns_new, x='Index', y=['Index Return', 'Portfolio Return'], title='Portfolio vs Index Returns: 2020')
fig.update_layout(
    legend=dict(
        orientation="h",  # Set orientation to horizontal
        yanchor="bottom",  # Anchor the legend to the bottom
        y=1.02  # Adjust the position of the legend
    )
)
fig.show()

In [24]:
data_MIP = pd.read_csv('MIPApproach_T-3600.csv')
data_MIP['Delta'] = data_MIP['2020_return_error'] - data_MIP['2019_return_error']
fig = px.line(data_MIP, x='m', y=['2019_return_error', '2020_return_error'], title='Error Comparison: 2019 vs 2020')
fig.show()

#### Combined Analysis

In [56]:
data_combined = data_MIP.merge(data_2_step, left_on='m', right_on='m', how='inner', suffixes=['_MIP', '_2_step'])
fig = px.line(data_combined, x='m', y=['2020_return_error_MIP', '2020_return_error_2_step'], title='Error Comparison: 2-Step vs MIP')
fig.update_layout(
    legend=dict(
        orientation="h",  # Set orientation to horizontal
        yanchor="bottom",  # Anchor the legend to the bottom
        y=1.02  # Adjust the position of the legend
    )
)
fig.show()

In [26]:
MIP_20 = pd.read_csv('MIPApproach_T-20.csv')
MIP_3600 = pd.read_csv('MIPApproach_T-3600.csv')
combined_data = MIP_3600.merge(MIP_20, how='inner', left_on='m', right_on='m', suffixes=['_3600', '_20'])

fig = px.line(combined_data, x='m', y=['2020_return_error_3600', '2020_return_error_20'], title='MIP Error Comparison: 20 vs 3600')
fig.show()